In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
import numpy as np
plt.style.use(['seaborn-colorblind','paper'])
plt.style.use('classic')
import Cosmology as Cosmo

In [2]:
def dNdz_LSST(bin_num,dn_filename = 'dndz_LSST_i27_SN5_3y'):
    bins,big_grid,res   = pickle.load(open(dn_filename+'_extrapolated.pkl','r'))
    mbin                = bins[bin_num]
    zbin                = big_grid
    nbin                = res[bin_num]
    norm                = simps(nbin,zbin)
    dndz                = interp1d(zbin, nbin/norm, kind='linear',bounds_error=False,fill_value=0.)
    print 'using z-bin', mbin
    return norm

In [ ]:
"""------------settings----------------"""
bin_num     = 0

fsky        = 0.5

L = np.arange(100,3000)

params      = Cosmo.Planck2015
tag         = params[0]['name']
nl          = True

N0_path     ='/home/nessa/Documents/Projects/LensingBispectrum/CMB-nonlinear/outputs/N0files/'
spec_path   ='/home/nessa/Documents/Projects/LensingBispectrum/CMB-nonlinear/outputs/power_spectra/'
ellpath     ='/home/nessa/Documents/Projects/LensingBispectrum/CMB-nonlinear/outputs/ells/'
ellfile     ='ell_ang_equilat_Lmin10_Lmax10000_lmin10_lmax8000_lenL200_lenl220_lenang200_0e+00.pkl'

#bispectrum code
config_kk = 
config_kg = 
config_gg = 
Lmin = 1
Lmax = 3000

thetaFWHMarcmin = 1.4 #beam FWHM
noiseUkArcmin   = 6. #eval(sys.argv[1]) #Noise level in uKarcmin
l_max_T         = 3000
l_max_P         = 5000
l_min           = 50
L_max           = 6000 #for l integration
L_min           = 1
TCMB            = 2.7255e6
div             = False #divide EB by factor of 2.5

if l_max_T!=l_max_P:
    lmax='mixedlmax_%d_%d'%(l_max_T,l_max_P)
else:
    lmax=str(l_max_T)

if nl:
  nl_='_nl'
else:
  nl_=''

In [ ]:
filename = N0_path+'%s_N0_%s_%d_%d%d_%s%s.pkl'%(tag,lmax,l_min,10*noiseUkArcmin,10*thetaFWHMarcmin,no_div,nl_)
print N0_path+'%s_N0_%s_%d_%d%d_%s%s.pkl'%(tag,lmax,l_min,10*noiseUkArcmin,10*thetaFWHMarcmin,no_div,nl_)
Ls,AL = pickle.load(open(filename,'r'))

In [ ]:
#percent accuracy, should be sufficient for this
tag = tag+"_"+"LSSTbin"+str(bin_num)
config_kk+='_full_'+tag
config_kg+='_full_'+tag
config_gg+='_full_'+tag

ll, l, theta = pickle.load(open(ellpath+ellfile,'r'))
clgg = np.load(spec_path+"CL_%s_Lmin%d-Lmax%d_%s"%(config_gg,Lmin,Lmax,'halofit'))
clgg = np.interp(L,ll,clgg)
clkg = np.load(spec_path+"CL_%s_Lmin%d-Lmax%d_%s"%(config_kg,Lmin,Lmax,'halofit'))
clkg = np.interp(L,ll,clkg)
clkk = np.load(spec_path+"CL_%s_Lmin%d-Lmax%d_%s"%(config_kk,Lmin,Lmax,'halofit'))
clkk = np.interp(L,ll,clkk)

In [ ]:
n0  = np.interp(L,Ls,1./4.*L**2*AL)
n_bar = dNdz_LSST(bin_num)*(180*60/np.pi)**2

noise_kk      = np.sqrt((2./(2.*L+1.)/fsky))*(cl_kk+n0)
noise_gg      = np.sqrt((2./(2.*L+1.)/fsky))*(cl_gg+1./n_bar)
noise_kg      = 1./(2*L+1)/fsky*((clkk+n0)*(clgg+1./n_bar)+clkg**2)
noise_kg      = np.sqrt(noise_kg)

pickle.dump([clgg,clkg,clkk,noise_gg,noise_kg,noise_kk],open('Gaussian_variances_CMB-S4_bin%s_%s_%s.pkl'%(red_bin,tag,dn_filename),'w'))

In [ ]:
#make plots to test this